In [27]:
# @title Preparation
#Drive Link materiale: https://drive.google.com/drive/folders/10T3u5qfomOPUeXfHPEzeA3GRiT-ZqgBk?usp=sharing
!pip install tensorflow-gpu==1.15
!pip install scikit-learn==0.20.1

!pip install -q keras-bert keras-rectified-adam
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
import tensorflow as tf
import keras
from keras_radam import RAdam
from keras_bert import get_custom_objects
import numpy as np
from keras_bert import Tokenizer
import pandas as pd
import tensorflow.keras.backend as K
import sys
from sklearn.metrics import classification_report
from google.colab import drive

In [0]:
# @title Constants
np.random.seed(42)
SEQ_LEN = 128
BATCH_SIZE = 8
EPOCHS = 5
LR = 1e-5

In [0]:
# @title Environment
import os
pretrained_path = '/content/drive/My Drive/codiesp/'
config_path = os.path.join(pretrained_path, 'config.json')
checkpoint_path = os.path.join(pretrained_path, 'model.ckpt-2000000')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [31]:
# @title Load Basic Model
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
from run_classifier import *
import modeling
import optimization
import tokenization

import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


Cloning into 'bert_repo'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 300.28 KiB | 4.00 MiB/s, done.
Resolving deltas: 100% (185/185), done.



In [32]:
# @title Load Data
!pip install category_encoders==1.3.0
import joblib
import pandas as pd
from keras import Sequential
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import random
from keras.layers import Input
import keras
from keras.layers import Conv1D , Embedding
from keras.layers import Dropout
from keras.layers import MaxPool1D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import GlobalMaxPool1D
from keras.layers import Bidirectional
import category_encoders as ce

from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention

def remove_symbol(s):
    s = s.replace(",", "")
    s = s.replace(".", "")
    s = s.replace(";", "")
    s = s.replace(":", "")
    s = s.replace("_", "")
    s = s.replace("+", "")
    s = s.replace("ª", "")
    s = s.replace("-", "")
    s = s.replace("<", "")
    s = s.replace(">", "")
    s = s.replace("!", "")
    s = s.replace("?", "")
    s = s.replace("(", "")
    s = s.replace(")", "")
    s = s.replace("[", "")
    s = s.replace("]", "")
    s = s.replace("'", "")
    s = s.replace("0", "")
    s = s.replace("1", "")
    s = s.replace("2", "")
    s = s.replace("3", "")
    s = s.replace("4", "")
    s = s.replace("5", "")
    s = s.replace("6", "")
    s = s.replace("7", "")
    s = s.replace("8", "")
    s = s.replace("9", "")
    s = s.replace("%", "")
    s = s.strip()
    s = s.lower()
    return s

# Rimozione delle stopword
def clar_text(text):
    t = remove_symbol(str(text).strip().lower())
    tokens = list(str(text).lower().split(" "))
    for z in range(0, len(stop_word)):
        if stop_word[z] in tokens:
            while stop_word[z] in tokens:
                tokens.remove(str(stop_word[z]))

    tt = ""
    for it in tokens:
      tt = tt +" "+it
    return tt

def _pad(input_ids, max_seq_len):
    x = []
    input_ids = input_ids[:min(len(input_ids), max_seq_len - 2)]
    input_ids = input_ids + [0] * (max_seq_len - len(input_ids))
    return np.array(input_ids)

#LOADING DATASET
df = pd.read_csv('/content/drive/My Drive/codiesp/TestSet.csv')
df = df[['id', 'Desc']]
# df = df[pd.notnull(df['desc'])]
#print(df.head(10))
print(df.shape)

df.index = range(df.shape[0])
print("Parole: " + str(df['Desc'].apply(lambda x: len(x.split(' '))).sum()))  # ci sono circa 211456 parole

# rimozione SOLO dei simboli (nessuno stemming e nessuna rimozione delle stopword)
df['Desc'] = df['Desc'].apply(remove_symbol)
#print(df.head(10))

# Acquisizione delle stop word
file_stopw = open("/content/drive/My Drive/codiesp/stop_word.pck", "rb")
stop_word = pickle.load(file_stopw)
df['Desc'] = df['Desc'].apply(clar_text)

#suddivisione train_test
#train, test = train_test_split(df, test_size=0.3, random_state=42)
test = df
#prepare class encoder
#le = ce.OneHotEncoder(return_df=False, impute_missing=False, handle_unknown="ignore")
#labels = le.fit(list(df['Code']))
mapa = [0,1]

labels_map = [0,1]
#i = 0
#for a in mapa:
#    labels_map.append(a)
#print(labels_map)

#Tokenization
#Inizialize the tokenizer
tokenizer = tokenization.FullTokenizer(vocab_path, do_lower_case=True)
#indices_train = []
indices_test = []

#for text in train['Desc']:
#  tk = tokenizer.tokenize(text)
#  tokens = ["[CLS]"] + tk + ["[SEP]"]
#  token_ids = tokenizer.convert_tokens_to_ids(tokens)
#  token_ids = _pad(token_ids,SEQ_LEN)
#  indices_train.append(token_ids)

for text in test['Desc']:
  tk = tokenizer.tokenize(text)
  tokens = ["[CLS]"] + tk + ["[SEP]"]
  token_ids = tokenizer.convert_tokens_to_ids(tokens)
  token_ids = _pad(token_ids,SEQ_LEN)
  indices_test.append(token_ids)

#indices_train = [indices_train, np.zeros_like(indices_train)]
indices_test= [indices_test, np.zeros_like(indices_test)]
ids = test['id']
print(ids)
#train_labes_indexes = []
#train_labels = train['Code']
#for label in train_labels:
#  if(label =='emp'):
#    train_labes_indexes.append(0)
#  else:
#    train_labes_indexes.append(1)


     |████████████████████████████████| 61kB 2.1MB/s 
(53220, 2)
Parole: 1131000

0        S1137-66272016000300011-1
1        S1137-66272016000300011-1
2        S1137-66272016000300011-1
3        S1137-66272016000300011-1
4        S1137-66272016000300011-1
                   ...            
53215    S1130-01082017000800012-1
53216    S1130-01082017000800012-1
53217    S1130-01082017000800012-1
53218    S1130-01082017000800012-1
53219    S1130-01082017000800012-1
Name: id, Length: 53220, dtype: object


In [33]:
!pip install joblib
import joblib

labels_map = joblib.load('/content/drive/My Drive/codiesp/labels_map_bert_only_01.joblib')

In [34]:
bert = load_trained_model_from_checkpoint(
    config_file=config_path,
    checkpoint_file=checkpoint_path,
    training=True,
    trainable=True,
    seq_len=128
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# @title Build Custom Model

inputs = bert.inputs[:2]
dense = bert.get_layer('NSP-Dense').output
dense1 = keras.layers.Dense(units=1000, activation='tanh') (dense)
outputs = keras.layers.Dense(units=2, activation='softmax')(dense1)
modelk = keras.models.Model(inputs, outputs)
modelk.compile(
    RAdam(lr=LR),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

In [36]:
# @title Initialize Variables
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# @title Fit

#filepath="/content/drive/My Drive/codiesp/bert_only_01.{epoch:05d}-{val_loss:.5f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_sparse_categorical_accuracy', verbose=1, save_best_only=True, mode='max')

#    checkpoint
#callbacks_list = [
#]

#modelk.fit(
#    indices_train,
#    train_labes_indexes,
#    epochs=50,
#    batch_size=32,
#    validation_split = 0.10,
#    callbacks=callbacks_list
#)

In [0]:
modelk.load_weights('/content/drive/My Drive/codiesp/bert_only_01.00003-0.14749.hdf5')

In [39]:
# @title Predict
predicts = modelk.predict(indices_test, verbose=True)
print(predicts[0])

53220/53220 [==============================] - 843s 16ms/step
[0.16253373 0.83746624]


In [40]:
res_encoded = []
for a in predicts:
    val = a.argmax()
    res_encoded.append(labels_map[val])

print(res_encoded)
#res_labels = le.inverse_transform(DataFrame.res_encoded)

#test_labels = test['Code']
#test_labes_indexes = []
#for label in test_labels:
#  if label == 'emp':
#    test_labes_indexes.append(0)
#  else:
#    test_labes_indexes.append(1)



#print('Testing accuracy %s' % accuracy_score(test_labes_indexes, res_encoded))
#print('Testing F1 score: {}'.format(f1_score(test_labes_indexes, res_encoded, average='weighted')))


[1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 

In [0]:
#from sklearn.metrics import classification_report

In [0]:
#print(classification_report(test_labes_indexes, res_encoded,digits=5))

#Cap_Classifier

In [0]:
# Tokenizzazione e rimozione delle stopword
def tokenize_text(text):
    tokens = list(str(text).lower().split(" "))
    for z in range(0, len(stop_word)):
        if stop_word[z] in tokens:
            while stop_word[z] in tokens:
                tokens.remove(str(stop_word[z]))
    return tokens

In [44]:
#Estraggo elementi di cui si crede ci sia la patologia
left_to_classify = []
#left_to_classify_labels = []
left_to_classify_ids = []
res_encoded_original = res_encoded.copy()

print(test.head(10))
test_desc = test['Desc']
test_bal = test['id']

print(test_bal[0])
i = 0
for item in res_encoded:
  if item == 1:
    left_to_classify.append(test_desc[i])
    left_to_classify_ids.append(test_bal[i])
  i = i+1

print(len(left_to_classify))
print(len(left_to_classify_ids))

joblib.dump(left_to_classify,'/content/drive/My Drive/codiesp/left_to_classify_Task1_run2.vec')
joblib.dump(left_to_classify_ids,'/content/drive/My Drive/codiesp/left_to_classify_ids_Task1_run2.vec')

                          id                                               Desc
0  S1137-66272016000300011-1   varón  años consultó dolor tumefacción borde ...
1  S1137-66272016000300011-1                     refería antecedente traumático
2  S1137-66272016000300011-1        dolor carácter espontáneo inicio progresivo
3  S1137-66272016000300011-1   dicho dolor inicialmente cedía aine increment...
4  S1137-66272016000300011-1   exploración física apreció tumefacción mal de...
5  S1137-66272016000300011-1                 enrojecimiento aumento calor local
6  S1137-66272016000300011-1                  movilidad muñeca dedos conservada
7  S1137-66272016000300011-1   radiografía simple mostraba lesión ósea radio...
8  S1137-66272016000300011-1   completó estudio gammagrafía ósea tac resonan...
9  S1137-66272016000300011-1   gammagrafía ósea tres fases realizada tecneci...
S1137-66272016000300011-1
28436
28436


['/content/drive/My Drive/codiesp/left_to_classify_ids_Task1_run2.vec']

In [45]:
from sklearn.svm import SVC
from gensim.models import Doc2Vec
import warnings
warnings.filterwarnings("ignore")

left_to_classify_res= None

for i in range(0, 21):
   # model_dmm = Doc2Vec(dm=1, dm_mean=1, window=10, negative=5, min_count=1, workers=5, alpha=0.065,min_alpha=0.065)
    model_dmm = Doc2Vec.load('/content/drive/My Drive/codiesp/models_BinCaps/'+'Model_DMM_BinClassCap' + str(i) + '.bin')
    transf = lambda t: model_dmm.infer_vector(t, steps=20)
    dvc_vectors = np.array([transf(tokenize_text(xi)) for xi in left_to_classify]) 
    #print(dvc_vector)
    model = joblib.load('/content/drive/My Drive/codiesp/models_BinCaps/'+'Model_SVM_BinClassCap' + str(i) + '.bin')
    res = model.predict_log_proba(dvc_vectors)
    #print(res)
    extract = lambda t: [t[1]]
    arr_res = np.array([extract(xi) for xi in res])
    print(arr_res)
    #arr_res_sent.append(res[0][1])
    if left_to_classify_res is None:
      left_to_classify_res = arr_res
    else:
      left_to_classify_res = np.concatenate((left_to_classify_res,arr_res),axis=1)
      #print(left_to_classify_res)

print(left_to_classify_res)




[[-1.20081424]
 [-4.35528661]
 [-0.27093966]
 ...
 [-4.65258383]
 [-7.49001567]
 [-4.01790513]]
[[-1.55198917]
 [-7.68420263]
 [-6.12539836]
 ...
 [-5.48896386]
 [-2.74208899]
 [-0.8317873 ]]
[[ -4.6768503 ]
 [ -3.8824194 ]
 [ -5.69128154]
 ...
 [ -8.00672945]
 [ -5.26091032]
 [-14.6663094 ]]
[[-4.94007273]
 [-4.39696328]
 [-4.70505833]
 ...
 [-9.25468982]
 [-4.90257248]
 [-5.61831028]]
[[ -3.09453113]
 [ -0.733596  ]
 [ -0.97457285]
 ...
 [ -6.7007578 ]
 [-10.1276024 ]
 [ -8.13070139]]
[[-5.21497006]
 [-7.99488814]
 [-6.13332535]
 ...
 [-6.18404705]
 [-6.52203768]
 [-7.82375852]]
[[-7.51692074]
 [-4.45530681]
 [-6.75176651]
 ...
 [-8.15910951]
 [-6.17217265]
 [-5.79920685]]
[[-6.32044854]
 [-6.28297936]
 [-6.05692866]
 ...
 [-6.15082576]
 [-6.25132578]
 [-6.44909109]]
[[-6.34397698]
 [-7.37249829]
 [-5.43112282]
 ...
 [-8.20321854]
 [-7.7955121 ]
 [-6.29054641]]
[[-6.63480242]
 [-9.60077566]
 [-7.31418445]
 ...
 [-6.84275303]
 [-7.69455747]
 [-9.34332457]]
[[ -6.27289939]
 [ -4.802519

In [0]:
#left_to_classify_res[2]

#for item in left_to_classify_res:
#  for val in item:
#    if val > -1.5:
#      print(val)
#  print('----------')

In [47]:
c = {}
p = {}
r = {}

for i in range(0,21):
  key = 'c'+str(i)
  value = []
  c[key] = value
  key = 'p'+str(i)
  value1 = []
  p[key] = value1
  key = 'r'+str(i)
  value2 = []
  r[key] = value2

print(c)
print(p)
print(r)


{'c0': [], 'c1': [], 'c2': [], 'c3': [], 'c4': [], 'c5': [], 'c6': [], 'c7': [], 'c8': [], 'c9': [], 'c10': [], 'c11': [], 'c12': [], 'c13': [], 'c14': [], 'c15': [], 'c16': [], 'c17': [], 'c18': [], 'c19': [], 'c20': []}
{'p0': [], 'p1': [], 'p2': [], 'p3': [], 'p4': [], 'p5': [], 'p6': [], 'p7': [], 'p8': [], 'p9': [], 'p10': [], 'p11': [], 'p12': [], 'p13': [], 'p14': [], 'p15': [], 'p16': [], 'p17': [], 'p18': [], 'p19': [], 'p20': []}
{'r0': [], 'r1': [], 'r2': [], 'r3': [], 'r4': [], 'r5': [], 'r6': [], 'r7': [], 'r8': [], 'r9': [], 'r10': [], 'r11': [], 'r12': [], 'r13': [], 'r14': [], 'r15': [], 'r16': [], 'r17': [], 'r18': [], 'r19': [], 'r20': []}


In [48]:
k = 0

for it in left_to_classify_res:

  positions = []
  for index in range(0,len(it)):
    if it[index] > -1.5:
      positions.append(index)
    
  if len(positions) == 0:
    index = it.argmax()
    positions.append(index)
  
  for val in positions:
    index = val
    
    data = c['c'+str(index)]
    data.append(left_to_classify[k]);
    c['c'+str(index)] = data
    
    data1 = p['p'+str(index)]
    data1.append(k);
    p['p'+str(index)] = data1

  k = k+1

#print(c)
print(p)

{'p0': [0, 2, 3, 14, 15, 20, 26, 32, 33, 40, 41, 42, 45, 47, 48, 57, 77, 78, 84, 92, 95, 97, 101, 105, 114, 122, 156, 158, 160, 166, 167, 170, 175, 177, 182, 185, 192, 193, 194, 197, 198, 202, 204, 206, 207, 209, 212, 218, 222, 225, 230, 233, 236, 240, 242, 244, 245, 246, 250, 252, 253, 257, 262, 267, 268, 269, 270, 271, 278, 281, 286, 290, 292, 297, 300, 301, 307, 312, 314, 319, 320, 321, 324, 325, 333, 339, 350, 354, 355, 361, 363, 364, 365, 367, 373, 376, 382, 386, 387, 391, 392, 407, 410, 411, 412, 413, 417, 419, 420, 430, 432, 436, 441, 447, 465, 467, 468, 469, 471, 473, 481, 482, 483, 485, 487, 492, 501, 506, 534, 540, 547, 555, 559, 560, 569, 570, 571, 579, 585, 586, 595, 596, 610, 616, 619, 634, 637, 638, 640, 652, 653, 659, 666, 669, 670, 671, 688, 690, 702, 703, 710, 713, 722, 729, 738, 762, 764, 766, 768, 771, 775, 779, 785, 790, 796, 801, 802, 810, 812, 818, 826, 829, 835, 836, 838, 839, 842, 844, 850, 852, 858, 861, 867, 869, 870, 874, 877, 881, 895, 903, 904, 928, 929, 93

#CLASSIFICATION pathologies with their numbers


In [49]:
t = joblib.load("/content/drive/My Drive/codiesp/lstm_fasttext__tokenizer_total.vec")
embedding_matrix = joblib.load('/content/drive/My Drive/codiesp/lstm_fasttext__embedding_matrix_medical_total.vec')
z = 0
from collections import OrderedDict 

for z in range(0,21):
  to_classify = c['c'+str(z)]
  #padding text
  encoded_left_to_classify = t.texts_to_sequences(to_classify)

  max_length = 64
  padded_left_to_classify = pad_sequences(encoded_left_to_classify, maxlen=max_length, padding='post')
  #print(padded_left_to_classify[76])

  le = joblib.load("/content/drive/My Drive/codiesp/modelsLE/"+str(z)+"_le.joblib")
  # define the model
  vocab_size = len(t.word_index) + 1
  input = Input(shape=(64,))
  m = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=64, trainable=False) (input)
  bi = Bidirectional(LSTM(64, activation ='tanh', return_sequences = True, dropout=0.3)) (m)

  aa = SeqSelfAttention(attention_activation='tanh') (bi)
  aa = Conv1D(128,5, activation ='relu' ) (aa)
  aa = MaxPool1D(2) (aa)
  aa = Dropout(0.2) (aa)

  added = keras.layers.Concatenate(axis=1)([aa,bi])

  ff = GlobalMaxPool1D() (added)
  ff = Dense(4000)(ff)
  ff = Dropout(0.3) (ff)
  ff =Dense(len(le.category_mapping[0]['mapping']), activation='softmax') (ff)

  modelz = keras.models.Model(inputs=[input], outputs=[ff])

  #modelz.summary(line_length=100)
  modelz.compile (loss='categorical_crossentropy' , optimizer='adam' , metrics=[ 'accuracy'] )
  modelz.load_weights('/content/drive/My Drive/codiesp/modelsCaps/'+str(z)+'.hdf5')
  #predict
  res = modelz.predict(padded_left_to_classify)

  #map = le.category_mapping[0]['mapping']
  #print(map)

  #labels_map = []
  #i = 0
  #for a,b in map:
  #    labels_map.append(a)

  max_vals = {}
  extracted_res = []
  extractede_ids = []
  map = le.category_mapping[0]['mapping']
  labels_map = []
  i = 0
  for a,b in map:
      labels_map.append(a)

  j =0

  l_ids = p['p'+str(z)]
  previous_id = l_ids[0]
  for result in res:
    id = l_ids[j]
    if id != previous_id:
      od = OrderedDict(max_vals) 
      #print(str(id)+' '+str(od))
      extracted_res.append(od)
      max_vals = {}
      previous_id = id

    i = 0
    for a in result:
      if a > 0.10:
        max_vals[str(a)] = labels_map[i]
      i = i+1
    if len(max_vals) == 0:
      v = result.max()
      index = result.argmax()
      max_vals[str(v)] = labels_map[index]

    j = j+1

  #Last item
  od = OrderedDict(max_vals) 
  extracted_res.append(od)

  print(extracted_res)

  #res_encoded_ins = []
  #for a in res:
  #    val = a.argmax()
  #    res_encoded_ins.append(labels_map[val])

  #print(res_encoded_ins)
  r['r'+str(z)] = extracted_res

print(r)



Output hidden; open in https://colab.research.google.com to view.

In [50]:
joblib.dump(res_encoded_original,'/content/drive/My Drive/codiesp/task1/res_encoded_res.joblib')
joblib.dump(test['id'],'/content/drive/My Drive/codiesp/task1/ids_res.joblib')
joblib.dump(c,'/content/drive/My Drive/codiesp/task1/c_res.joblib')
joblib.dump(p,'/content/drive/My Drive/codiesp/task1/p_res.joblib')
joblib.dump(r,'/content/drive/My Drive/codiesp/task1/r_res.joblib')

['/content/drive/My Drive/codiesp/task1/r_res.joblib']

In [0]:
import joblib
res_encoded = joblib.load('/content/drive/My Drive/codiesp/task1/res_encoded_res.joblib')
c = joblib.load('/content/drive/My Drive/codiesp/task1/c_res.joblib')
p= joblib.load('/content/drive/My Drive/codiesp/task1/p_res.joblib')
r = joblib.load('/content/drive/My Drive/codiesp/task1/r_res.joblib')

#Merge back the results


In [52]:
from collections import defaultdict

dica = defaultdict()
print(r)
from collections import OrderedDict
#dic = {}
for i in range(0,21):
  pos = p['p'+str(i)]
  res = np.array(r['r'+str(i)])

  for k in range(0,len(res)):
    pa = str(pos[k])
    for key,val in res[k].items():
      try:
        dica[pa].append(val)
      except KeyError: # If it fails, because there is no key
        dica[pa] = [val]
      #try:
      #  arr = dic[pa]
      #  arr.append(val)
      #  dic[pa] = arr
      #except:
      #  dic[pa] = [val]

#print(dic)
oda=OrderedDict(dica)

#for key, value in oda.items(): 
#    print(key, value) 
print(oda['28436'])

Output hidden; open in https://colab.research.google.com to view.

In [0]:
final_res = []
file = open('1.task1_run1_bert_01_single_class_deep_net.tsv','w+')


i = 0
for item in res_encoded_original:
  if item == 0:
    final_res.append('emp')
  else:
    id = ids[i]
    arr = oda[str(i)]
    for el in arr:
      file.write(str(id)+'\t'+str(el)+'\n')
      file.flush()
    i = i+1



file.close()

In [0]:
print('Testing accuracy %s' % accuracy_score(test_bal, final_res))
print('Testing F1 score: {}'.format(f1_score(test_bal, final_res, average='weighted')))
print(classification_report(test_bal, final_res,digits=5))

In [0]:
print(final_res)